In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import undetected_chromedriver as uc

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    # Set the user data directory to store the session
    options.add_argument(r'--user-data-dir=C:\Users\Irfan Ullah\Desktop\sessions')

    driver = uc.Chrome(options=options)
    return driver
driver = init_driver()
driver.get("https://www.facebook.com/groups/383691761755058/members")



# # except NoSuchElementException:
# #     print("[ERROR] Member elements not found on the page.")

# def click_about_section(driver):
#     # Wait for the "About" section link to be clickable and click it
#     about_section = WebDriverWait(driver, 10).until(
#         EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'about')]"))
#     )
#     about_section.click()
#     time.sleep(5)  # Wait for the About section to load

#     print("[INFO] Clicked on the 'About' section.")

# def check_overview_section(driver):
#     try:
#         smooth_scroll(driver, scroll_amount=200)
#         print("Scrolled down slightly.")
        
#         about_button = WebDriverWait(driver, 20).until(
#             EC.element_to_be_clickable((By.XPATH, "//span[text()='About']"))
#         )
#         scroll_to_element_smoothly(driver, about_button)
#         print("Scrolled to the 'About' button.")
        
#         driver.execute_script("arguments[0].click();", about_button)
#         time.sleep(7)
#         print("About button clicked successfully.")

#         smooth_scroll(driver, scroll_amount=200)
#         studies_xpath = "//div[contains(@class, 'x13faqbe') and contains(@class, 'x78zum5') and contains(@class, 'xdt5ytf')]//span[contains(text(), 'Studies')]"
#         studies_text = extract_text_from_element(driver, studies_xpath)
#         print(studies_text)

#         if studies_text:
#             if "Studies " in studies_text:
#                 print("Currently enrolled in a program.")
#                 print("He/she is a student and we can message him/her.")
#         else:
#             print("Not a Student.")

#     except Exception as e:
#         print(f"Error: {e}")
#     finally:
#         driver.quit()




# Click on the "About" section
# click_about_section(driver)

# Specify the university to check for
# univer = "Abertay University"

# Check the overview section for the university
# check_overview_section(driver)

  # Make sure to quit the driver after the script is done


univer = "Abertay University"
def match_and_click_people(driver, univer):
    try:
        # Scroll and load more people
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down to the bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for the page to load

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Find all people in the "People" tab
        people_items = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-visualcompletion='ignore-dynamic'] a"))
        )

        # Debugging information
        print(f"[INFO] Found {len(people_items)} people items.")

        # Extract links of matched people
        matched_people_links = []
        for item in people_items:
            try:
                title_element = item.find_element(By.XPATH, "//span[@class='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1pg5gke xvq8zen xo1l8bm xi81zsa x1yc453h']")
                title = title_element.text.strip()
                print(f"[DEBUG] Found title: {title}")
                for index, row in univer.iterrows():
                    university_name = row['university']
                    if university_name.lower() in title.lower():
                        link = item.get_attribute('href')
                        if link and isinstance(link, str):
                            matched_people_links.append(link)
                            break
                        else:
                            print("[ERROR] Invalid URL found for a person item.")
            except:
                
                print("[ERROR] Title element not found for a person item.")

        # Click on matched people links one by one
        visited_links = set()
        for link in matched_people_links:
            if link not in visited_links:
                visited_links.add(link)
                driver.get(link)
                time.sleep(5)  # Wait for the profile page to load
                print(f"[INFO] Clicked on the profile link: {link}")

                # Click on the "View Profile" link
                click_view_profile(driver)

                # Check the default overview section for "studies" or "studied"
                check_overview_section(driver, univer)

    except TimeoutException:
        print("[ERROR] Timeout while waiting for the people items to be clickable.")
    except NoSuchElementException:
        print("[ERROR] People items element not found.")
    except Exception as e:
        print(f"[ERROR] Error while matching and clicking on the people items: {e}")



IndentationError: unexpected indent (4270700538.py, line 33)

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import undetected_chromedriver as uc
import os

def login_if_needed(driver, username, password):
    global current_logged_in_username
    
    if not username or not password:
        print("[ERROR] Username or password is empty. Skipping login attempt.")
        return False
    try:
        driver.get("https://www.facebook.com/")
        time.sleep(5)  # Allow page to load

        # Check if the user is already logged in by looking for the "Home" link
        try:
            home_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[@aria-label='Home']"))
            )
            print(f"[INFO] Already logged in for {username}.")
            # Store the currently logged-in username in the global variable
            current_logged_in_username = username
            return True
        except TimeoutException:
            print("[INFO] Not logged in. Proceeding to login.")

        # If not logged in, proceed with login
        driver.get("https://www.facebook.com/login")
        time.sleep(5)  # Allow page to load

        email_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "email"))
        )
        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "pass"))
        )
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, "login"))
        )

        # Clear the input fields before sending keys
        email_input.clear()
        password_input.clear()

        email_input.send_keys(username)
        password_input.send_keys(password)
        login_button.click()
        time.sleep(5)  # Allow page to load

        # Check if login was successful by looking for the "Home" link again
        try:
            home_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[@aria-label='Home']"))
            )
            print(f"[INFO] Login successful for {username}.")
            # Store the currently logged-in username in the global variable
            current_logged_in_username = username
            return True
        except Exception as e: 
            print(f"[ERROR] Login failed for {username}: {e}")
            return False
    except Exception as e:
        print(f"[ERROR] Login failed for {username}: {e}")
        return False
def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    # Set the user data directory to store the session
    sessions_dir = os.path.join(os.getcwd(), "sessions")
    if not os.path.exists(sessions_dir):
        os.makedirs(sessions_dir)
        print(f"[INFO] Created sessions directory at: {sessions_dir}")
    
    options.add_argument(f'--user-data-dir={sessions_dir}')

    driver = uc.Chrome(options=options)
    return driver
driver = init_driver()
driver.get("https://www.facebook.com/groups/383691761755058/user/61553645454309/")

def message(driver):
    try:
        time.sleep(10)
        # Find the message button
        message_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[@class='x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft'][normalize-space()='Message']"))
        )
        message_button.click()
        print("clicked")
        time.sleep(5)  # Wait for the message window to load

        # message.click()
        # time.sleep(5)  # Wait for the message window to load

    except Exception as e:
        print(f"[ERROR] Error while clicking on the message button: {e}")   

message(driver) 



clicked
